In [1]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import urllib.request
import io
import numpy as np

In [2]:
def download_and_read_data():
    """
        Tai va doc du lieu tu mang xa hoi Facebook (tu SNAP)
    """
    url = "https://snap.stanford.edu/data/facebook_combined.txt.gz"
    response = urllib.request.urlopen(url)
    data = response.read()

    # Giai nen file
    df = pd.read_csv(io.BytesIO(data),
                    compression='gzip',
                    sep=" ",
                    names=["source", "target"])
    return df

In [3]:
def calculate_all_metrics(G):
    """
        Tinh toan tat ca cac so do cua do thi
        Args:
            G: Do thi NetworkX
        Returns:
            dict: Dictionary chua cac so do cua do thi
    """
    metrics = {}
    # 1. Thong tin co ban
    metrics['nodes'] = G.number_of_nodes()
    # |V|: So luong dinh cua do thi

    metrics['edges'] = G.number_of_edges()
    # |E|: Slg canh do thi

    metrics['density'] = nx.density(G)
    # Mat tho do thi
    # Cong thuc: D = 2|E| / (|V|(|V|-1))
    # - |E|: so canh thuc te
    # - |V|(|V|-1): so canh toi da co the co trong do thi vo huong

    metrics['average_degree'] = np.mean([d for n, d in G.degree()])

    # 2. Degree metrics
    degrees = dict(G.degree())
    metrics['avg_degree'] = sum(degrees.values()) / len(degrees)
    # Degree trung binh
    # Cong thuc: <k> = (1/|V|) * Σ ki
    # - |V|: slg dinh
    # - ki: degree cua dinh i
    # Note: Trong do thi vo huong: <k> = 2|E|/|V|

    metrics['max_degree'] = max(degrees.values())
    metrics['degrees'] = degrees

    # 3. Degree Centrality
    dc = nx.degree_centrality(G)
    metrics['degree_centrality'] = {
        'values': dc,
        'max': max(dc.values()),
        'avg': sum(dc.values()) / len(dc),
        'node_max': max(dc, key=dc.get)
    }
    # Degree Centrality
    # Cong thuc: CD(v) = deg(v)/(|V|-1)
    # - deg(v): degree dinh v
    # - |V|-1: slg ket noi toi da co the co cua 1 dinh

    # 4. Betweenness Centrality
    bc = nx.betweenness_centrality(G)
    metrics['betweenness_centrality'] = {
        'values': bc,
        'max': max(bc.values()),
        'avg': sum(bc.values()) / len(bc),
        'node_max': max(bc, key=bc.get)
    }
    # Betweenness Centrality
    # Cong thuc: CB(v) = Σ (σst(v)/σst)
    # - σst: so duong di ngan nhat tu dinh s den dinh t
    # - σst(v): so duong di ngan nhat tu s den t di qua v
    # - Tong duoc tinh tren moi cap dinh s,t kha v

    # 5. Closeness Centrality
    cc = nx.closeness_centrality(G)
    metrics['closeness_centrality'] = {
        'values': cc,
        'max': max(cc.values()),
        'avg': sum(cc.values()) / len(cc),
        'node_max': max(cc, key=cc.get)
    }
    # Closeness Centrality
    # Cong thuc: CC(v) = (|V|-1) / Σ d(v,u)
    # - |V|-1: so dinh khac v
    # - d(v,u): do dai duong di ngan nhat tu v den u
    # - Tong duoc tinh tren moi dinh u khac v

    # 6. PageRank
    pr = nx.pagerank(G, alpha=0.85)
    metrics['pagerank'] = {
        'values': pr,
        'max': max(pr.values()),
        'avg': sum(pr.values()) / len(pr),
        'node_max': max(pr, key=pr.get)
    }
    # PageRank
    # Cong thuc: PR(v) = (1-d) + d * Σ (PR(u)/OutDegree(u))
    # - d: damping factor (thuong = 0.85)
    # - PR(u): PageRank cua cac dinh u ke voi v
    # - OutDegree(u): bac ra cua dinh u
    # - Tong duoc tinh tren moi dinh u ke voi v

    return metrics

In [4]:
def print_detailed_results(metrics):
    """
    In ket qua cua cac phep do
    """
    print("\n======== PHÂN TÍCH MẠNG XÃ HỘI ========= ")
    print("1. Thông tin cơ bản")
    print(f"- Số lượng nodes (người dùng): {metrics['nodes']}")
    print(f"- Số lượng cạnh (kết nối): {metrics['edges']}")
    print(f"- Mật độ: {metrics['density']}")
    print(f"- Degree trung bình: {metrics['average_degree']}")
    print(f"- Degree lớn nhất: {metrics['max_degree']}")

    centrality_measures = {
        'Degree Centrality': 'degree_centrality',
        'Betweenness Centrality': 'betweenness_centrality',
        'Closeness Centrality': 'closeness_centrality',
        'PageRank': 'pagerank'
    }

    print("2. Các số đo Centrality")

    for name, measure in centrality_measures.items():
        print(f"- {name}")
        print(f"- Giá trị lớn nhất: {metrics[measure]['max']:.4f}")
        print(f"- Giá trị trung bình: {metrics[measure]['avg']:.4f}")
        print(f"- Node có giá trị cao nhất: {metrics[measure]['node_max']}")

In [5]:
df = download_and_read_data()
df.head()
G = nx.from_pandas_edgelist(df, "source", "target")
metrics = calculate_all_metrics(G)
print_detailed_results(metrics)


======== PHÂN TÍCH MẠNG XÃ HỘI ========= 
1. Thông tin cơ bản
- Số lượng nodes (người dùng): 4039
- Số lượng cạnh (kết nối): 88234
- Mật độ: 0.010819963503439287
- Degree trung bình: 43.69101262688784
- Degree lớn nhất: 1045
2. Các số đo Centrality
- Degree Centrality
- Giá trị lớn nhất: 0.2588
- Giá trị trung bình: 0.0108
- Node có giá trị cao nhất: 107
- Betweenness Centrality
- Giá trị lớn nhất: 0.4805
- Giá trị trung bình: 0.0007
- Node có giá trị cao nhất: 107
- Closeness Centrality
- Giá trị lớn nhất: 0.4597
- Giá trị trung bình: 0.2762
- Node có giá trị cao nhất: 107
- PageRank
- Giá trị lớn nhất: 0.0076
- Giá trị trung bình: 0.0002
- Node có giá trị cao nhất: 3437


In [ ]:
def visualize_metrics(graph, degree_data, centrality_data, top_nodes):
    """
    Visualize network metrics using 2x2 subplots.

    Parameters:
    - graph: NetworkX graph
    - degree_data: Dictionary of degree distribution
    - centrality_data: Dictionary with keys ('centrality_1', 'centrality_2') for centrality comparisons
    - top_nodes: List of top nodes data for comparison
    """
    fig, axs = plt.subplots(2, 2, figsize=(12, 10))

    # Degree Distribution
    axs[0, 0].bar(degree_data.keys(), degree_data.values(), color='skyblue')
    axs[0, 0].set_title("Degree Distribution")
    axs[0, 0].set_xlabel("Degree")
    axs[0, 0].set_ylabel("Count")

    # Centrality Comparison
    axs[0, 1].scatter(
        centrality_data['centrality_1'].values(),
        centrality_data['centrality_2'].values(),
        alpha=0.7, c='coral'
    )
    axs[0, 1].set_title("Centrality Comparison")
    axs[0, 1].set_xlabel("Centrality 1")
    axs[0, 1].set_ylabel("Centrality 2")

    # Top Nodes Comparison
    axs[1, 0].bar(
        range(len(top_nodes)),
        [data[1] for data in top_nodes],
        tick_label=[data[0] for data in top_nodes],
        color='lightgreen'
    )
    axs[1, 0].set_title("Top Nodes Comparison")
    axs[1, 0].set_xlabel("Nodes")
    axs[1, 0].set_ylabel("Metric Value")

    axs[1, 1].text(0.5, 0.5, "Additional Metric Here",
                   ha='center', va='center', fontsize=12)
    axs[1, 1].set_title("Additional Metric Placeholder")
    axs[1, 1].axis('off')

    plt.tight_layout()
    plt.show()

# Đọc file thành dataframe
file_path = 'data/facebook_combined.txt'
df = pd.read_csv(file_path, sep=' ', header=None, names=['Node1', 'Node2'])

G = nx.from_pandas_edgelist(df, source='Node1', target='Node2')
degree_distribution = {deg: count for deg, count in enumerate(nx.degree_histogram(G)) if count > 0}
centrality_comparison = {
    'centrality_1': nx.betweenness_centrality(G),
    'centrality_2': nx.closeness_centrality(G)
}
top_nodes_comparison = sorted(nx.degree(G), key=lambda x: x[1], reverse=True)[:5]

visualize_metrics(G, degree_distribution, centrality_comparison, top_nodes_comparison)